In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

assert os.environ.get("DAGSTER_HOME"), (
    "The DAGSTER_HOME env var is not set so dagster won't be able to find the assets."
    "Set the DAGSTER_HOME env var in this notebook or kill the jupyter server and set"
    " the DAGSTER_HOME env var in your shell and relaunch jupyter."
)

In [3]:
import pudl
import pathlib
import yaml
from pudl.etl import *
import logging
import sys
import copy

pudl_settings is being depcrated in favor of environment variables PUDL_OUTPUT and PUDL_CACHE
pudl_settings is being depcrated in favor of environment variables PUDL_OUTPUT and PUDL_CACHE. For more infosee: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet directories are no longer being used. Make sure there is a single directory named 'output' at the root of your workspace. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
pudl_settings is being depcrated in favor of environment variables PUDL_OUTPUT and PUDL_CACHE
pudl_settings is being depcrated in favor of environment variables PUDL_OUTPUT and PUDL_CACHE. For more infosee: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.html
sqlite and parquet directories are no longer being used. Make sure there is a single directory named 'output' at the root of your workspace. For more info see: https://catalystcoop-pudl.readthedocs.io/en/dev/dev/dev_setup.htm

In [4]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

## Load pre harvesting assets
To avoid rerunning the extract and transform steps, this notebook loads the most recent pre harvested asset values. **If you are debugging the code changes in the harvesting step, make sure you rematerialize the EIA extract and transform assets for the same years prior to running the following cells.** If you are you are debugging pre harvesting asset code changes' affect on the harvesting process, rerun the pre harvesting asset using dagit then rerun the following cells.

In [5]:
from pudl.etl import defs
from pudl.helpers import convert_cols_dtypes, get_asset_group_keys

from dagster import AssetKey

pre_harvested_eia923_asset_names = get_asset_group_keys("pre_harvested_eia923_assets", default_assets)
pre_harvested_eia860_asset_names = get_asset_group_keys("pre_harvested_eia860_assets", default_assets)
pre_harvest_eia_asset_names = pre_harvested_eia860_asset_names + pre_harvested_eia923_asset_names

eia_transformed_dfs = {}
with defs.get_asset_value_loader() as loader:
    eia_transformed_dfs = {asset_name: loader.load_asset_value(AssetKey(asset_name)) for asset_name in pre_harvest_eia_asset_names}


# Do some final cleanup and assign appropriate types:
eia_transformed_dfs = {
    name: convert_cols_dtypes(df, data_source="eia")
    for name, df in eia_transformed_dfs.items()
}

# Remove the clean_ prefix from the table names.
eia_transformed_dfs = {
    table_name.replace("clean_", ""): df
    for table_name, df in eia_transformed_dfs.items()
}

Context impl SQLiteImpl.
Will assume non-transactional DDL.
Context impl SQLiteImpl.
Will assume non-transactional DDL.


2023-03-14 17:17:26 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_plants_eia860
2023-03-14 17:17:26 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_boiler_generator_assn_eia860
2023-03-14 17:17:26 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_utilities_eia860
2023-03-14 17:17:26 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_ownership_eia860
2023-03-14 17:17:26 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_generators_eia860
2023-03-14 17:17:27 -0800 - dagster - DEBUG - system - Loading file from: /Users/bendnorman/catalyst/dagster-pudl-work/dagster_home/storage/clean_fuel_receipts_costs_eia923
20

# Run harvest w/ debug=True

In [6]:
# we want to investigate the harvesting of the plants in this case...
entity = 'generators'
# create the empty entities df to fill up
entities_dfs = {}
entities_dfs, eia_transformed_dfs, col_dfs = (
    pudl.transform.eia.harvesting(
        entity, eia_transformed_dfs, entities_dfs, debug=True)
)

2023-03-14 17:17:34 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generators_eia860.operational_status_code


Recoding generators_eia860.operational_status_code


2023-03-14 17:17:34 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generators_eia860.prime_mover_code


Recoding generators_eia860.prime_mover_code


2023-03-14 17:17:34 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generators_eia860.energy_source_code_1


...

Recoding generators_eia860.startup_source_code_4


2023-03-14 17:17:36 [    INFO] catalystcoop.pudl.metadata.classes:1673 Recoding generators_eia860.data_maturity


Recoding generators_eia860.data_maturity


2023-03-14 17:17:59 [    INFO] catalystcoop.pudl.transform.eia:553 Average consistency of static generators values is 99.91%


Average consistency of static generators values is 99.91%


# Use `col_dfs` to explore harvested values

In [7]:
pmc = col_dfs['prime_mover_code']

In [8]:
pmc.prime_mover_code.unique()

<StringArray>
['GT', 'CT', 'ST', 'HY', 'IC', 'WT', 'CA', 'PV', 'BA', 'OT', 'BT', 'FC', 'CP', 'WS', 'CS', 'FW', 'PS', 'CC', 'CE']
Length: 19, dtype: string